# Part 1: Good code

##  Chapter 1: Safety
1장의 목적
* 오류가 덜 발생하는 코드를 만드는 것

### Item 1: 가변성 제한하기
- var 보다는 val
- mutable property 보다는 immutable property
- mutable object와 class보다는 immutable obj와 class
- 변경이 필요하다면 immutable 한 data class 의 copy
- Collection 에 상태를 저장해야 한다면, mutable 보단 read-only Collection
- 불필요한 변경 지점은 만들지 말 것
- mutable 객체를 외부로 노출하지 말 것

예외
- 효율성을 위해 immutable 보다 mutable 한 object 를 사용하는 것이 나을 때가 있

#### 코틀린에서 가변성 제한하기
* 읽기 전용 프로퍼티(val) 사용하기
* 가변 컬렉션과 읽기 전용 컬렉션 구분하기
* data class의 copy 사용하기

##### 읽기 전용 프로퍼티(val) 사용하기
읽기 전용(read only) 라고 해서 immutable 하진 않다.

In [3]:
var firstName = "길동"
var lastName = "홍"
val fullName
    get() = "$lastName$firstName"

println(fullName)
firstName = "길길"
println(fullName)

홍길동
홍길길


val 은 프로퍼티 레퍼런스 자체를 변경할 수는 없으므로 동기화 문제를 줄일 수 있다.

##### 가변 컬렉션과 읽기 전용 컬렉션 구분하기
immutable 에서 mutable 로 다운캐스팅 해야하는 경우엔 copy를 사용해라.

In [3]:
// 좋은 예
val list = listOf(1, 2, 3)
val mutableList = list.toMutableList()
mutableList.add(4)

true

In [1]:
// 나쁜 예
val list = listOf(1, 2, 3) // JVM에서 listOf 는 java 의 Arrays.ArrayList 를 반환

if (list is MutableList) { //  java List 의 add, set 을 메서드를 포함하고 있는 MutableList 로 변경 가능
    list.add(4) // Arrays.ArrayList 엔 add, set 이 구현되어 있지 않음
}

null
java.lang.UnsupportedOperationException
	at java.base/java.util.AbstractList.add(AbstractList.java:153)
	at java.base/java.util.AbstractList.add(AbstractList.java:111)
	at Line_1_jupyter.<init>(Line_1.jupyter.kts:5)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:122)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:48)
	at kotlin.script.experimental.jvm.BasicJvm

##### data class 의 copy 사용하기
data class 의 copy 는 특정 프로퍼티의 값이 변경된 새로운 인스턴스를 만들어준다.

In [3]:
data class User(
    val name: String,
    val surname: String
)

var user = User("길동", "홍")
user = user.copy(surname = "김")
println(user)

User(name=길동, surname=김)


#### 다른 종류의 변경 가능 지점
|              | `val MutableList` | `var List` |
|--------------|-------------------|------------|
| 변경 가능 지점     | MutableList 내부    | 프로퍼티 자체    |
| 멀티 스레드 안정성   | 나쁨                | 좋음         |
| Delegates 추적 | 불가능               | 가능         |

mutable property(`var List`) 가 제어하기  더 쉬움

property 와 collection 을 모두 mutable 하게 사용하진 말라
- `var MutableList`

In [8]:
import kotlin.concurrent.thread

var list = listOf<Int>() // var List 여도 코드를 잘못 작성하면 손실이 발생할 수 있음. 이 코드의 결과는 항상 1000 보다 작음
for (i in 1..1000) {
    thread {
        list = list + i
    }
}
Thread.sleep(1000)
print(list.size)

988

In [6]:
import kotlin.properties.Delegates

var names by Delegates.observable(listOf<String>()) { _, old, new ->
    println("Names changed form $old to $new")
}

names += "Fablo"
names += "Bill"

Names changed form [] to [Fablo]
Names changed form [Fablo] to [Fablo, Bill]


#### 변경 가능 지점 노출하지 말기
변경 가능한 프로퍼티에 대한 조회 메서드를 제공하려면, 가변성을 제한하기 위해 직접 노출시키지 말고 다음의 방법을 사용해라.
1. data class 의 copy
2. return 타입을 read only 타입으로 업캐스팅 